#### A. Configuración General.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "../constantes.ipynb"

#b. Constantes a definir por el usuari.
#i. Dataset de lectura (post-feature engineering).
dataset_con_fe = dataset_file_fe_all_1_limpieza
#ii. Información sobre rango temporal del modelo.
cantidad_meses_train = "all"
ventana = 1
#iii. Meses de train y test.
mes_train = mes_train_all_menos_1
mes_test = mes_test
#iv. Ruta del modelo guardado post-optimización de hiperparámetros.
modelo =  "{}".format(modelos_path) + 'lgbm_{}_{}_undersampling_limpieza.txt'.format(cantidad_meses_train,ventana)
#v. Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling_limpieza"

In [ ]:
#3. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#4. Importo el modelo.
model_lgb = lgb.Booster(model_file=modelo)

In [ ]:
#5. Importo el estudio de Optuna.
#g. Recogemos los parámetros probados con Optuna.

#ii. Creo el estudio.
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

#### B. Pre-procesamiento.

In [ ]:
#1. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)
#iii. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#2. Dividimos entre conjuntos de datos.
#a. Datos para entrenar todo el modelo final para Kaggle.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

#b. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)

#### C. Envíos para encontrar el punto de corte óptimo con 10 semillas.

In [ ]:
#1. Configuramos la cantidad de veces que vamos a subir cada curva de intentos (semillas).
cantidad_semillas = 10
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Entrenamos y predecimos Agosto.
#i. Recorremos diferentes semillas para entrenar.
i= 0
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla: {}.".format(semilla))
    #ii. Activamos la API para realizar envíos a Kaggle.
    api = KaggleApi()
    api.authenticate()
    #iii. Tomamos el mejor modelo y con eso entrenamos.
    print("--> Mejores parámetros...")
    best_iter = study.best_trial.user_attrs["best_iter"]
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }
    #iv. Adaptamos el dataset de entrenamiento.
    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)
    #v. Entrenamos el modelo de Light GBM con los mejores hiperparámetros y la semilla aleatoria.
    print("--> Comienza el entrenamiento...")
    model_lgb = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    #vi. Predecimos Agosto.
    predicciones = model_lgb.predict(X_test)
    #vii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    X_test['Probabilidad'] = predicciones
    #viii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
    tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
    #ix. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
    cortes = range(8500,13500,500)
    #x. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
    num_subida_kaggle = 1
    print("--> Comienzan los envíos a Kaggle...\n")
    for envios in cortes:
        #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
        tb_entrega['Predicted'] = 0
        tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
        resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
        
        print("Cantidad de clientes {}".format(envios))
        #2. Guardamos el archivo para Kaggle.
        nombre_archivo = "KLimp_{}_{}_EK_0{}_{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle,semilla) #DF con -X meses, Y ventana, número de intento, número de semilla.
        ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
        resultados.to_csv(ruta_archivo, index=False)
        
        num_subida_kaggle += 1
        i += 1
        
        #3. Envío a Kaggle.
        #a. Defino los parámetros claves.
        mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train} c/undersampling y limpieza. Semilla:{semilla},corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
        competencia = 'dm-ey-f-2024-segunda'
        #c. Subo la Submission.
        while i<=18:
                api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
                print("Submission successful!")
                break
        else:
            print("Esperamos 30 segundos...")
            time.sleep(30)
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            i= 0
        
    #d. Borramos la columna de predicciones anterior.
    X_test.drop(["Probabilidad"],axis=1,inplace=True)

In [ ]:
#3. Análisis de las subidas a Kaggle.
#i. Eligiendo el mejor punto de corte.
# Dados las 10 semillas corridas, y los 10 diferentes puntos de corte, el promedio nos indica que el punto de corte
# más óptimo para este modelo es X envíos. En tanto la distribución promedio del Público se asemeja a la
# distribución promedio del Privado, dicho punto de corte sería el mejor para la distribución promedio del Privado.

In [ ]:
print("Hasta acá.")

#### D. Envios a Kaggle para elegir luego el valor Promedio del punto de corte óptimo (estabilizar el modelo).

In [ ]:
#1. Configuramos la cantidad de veces que vamos a subir cada curva de intentos (semillas).
cantidad_semillas = 40
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Entrenamos y predecimos Agosto.
#i. Recorremos diferentes semillas para entrenar.
i= 0
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla: {}.".format(semilla))
    #ii. Activamos la API para realizar envíos a Kaggle.
    api = KaggleApi()
    api.authenticate()
    #iii. Tomamos el mejor modelo y con eso entrenamos.
    print("--> Mejores parámetros...")
    best_iter = study.best_trial.user_attrs["best_iter"]
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }
    #iv. Adaptamos el dataset de entrenamiento.
    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)
    #v. Entrenamos el modelo de Light GBM con los mejores hiperparámetros y la semilla aleatoria.
    print("--> Comienza el entrenamiento...")
    model_lgb = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    #vi. Predecimos Agosto.
    predicciones = model_lgb.predict(X_test)
    #vii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    X_test['Probabilidad'] = predicciones
    #viii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
    tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
    #ix. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
    cortes = range(8500,13500,500)
    #x. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
    num_subida_kaggle = 1
    print("--> Comienzan los envíos a Kaggle...\n")
    for envios in cortes:
        #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
        tb_entrega['Predicted'] = 0
        tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
        resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
        
        print("Cantidad de clientes {}".format(envios))
        #2. Guardamos el archivo para Kaggle.
        nombre_archivo = "K_{}_{}_EK_0{}_{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle,semilla) #DF con -X meses, Y ventana, número de intento, número de semilla.
        ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
        resultados.to_csv(ruta_archivo, index=False)
        
        num_subida_kaggle += 1
        i += 1
        
        #3. Envío a Kaggle.
        #a. Defino los parámetros claves.
        mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train} c/undersampling. Semilla:{semilla},corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
        competencia = 'dm-ey-f-2024-segunda'
        #c. Subo la Submission.
        while i<=18:
                api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
                print("Submission successful!")
                break
        else:
            print("Esperamos 30 segundos...")
            time.sleep(30)
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            i= 0
        
    #d. Borramos la columna de predicciones anterior.
    X_test.drop(["Probabilidad"],axis=1,inplace=True)